# <u>TP4: Équations différentielles partielles</u>

## par Antoine Boissinot  - -  François Gaudreault  - -  Béatrice Lessard-Hamel

## Présenté à M. Philippe Després

### Date de remise : 4 avril 2021

In [20]:
import matplotlib.pylab as plt
import numpy as np
import timeit
from scipy import constants

In [21]:
#Constants
hbar = constants.hbar #Planck's constant divided by 2*pi [m^2 kg / s]
M = constants.electron_mass #Mass of the eectron [kg]
L = 1e-08 #Length of the box [m]
x_0 = L/2 # [m]
sigma = 1e-10 #[m]
kappa = 5e10 #[m^-1]

## L'équation de Schrödinger et la méthode de Crank Nicolson

### Question 1
Le but de ce numéro est d'obtenir le vecteur $\mathbf{v}$ qui est donné par l'équation $\mathbf{B}\psi = \mathbf{v}$. Étant donné que la matrice $\mathbf{B}$ est une matrice tridiagonale, il est possible d'utiliser l'expression suivante pour obtenir les élément $v_i$ de $\mathbf{v}$: $$v_i = b_1 \psi_i + b_2(\psi_{i+1} + \psi_{i-1}).$$

In [53]:
#Paramters of the simulation

N = 1000 #Number of spatial slice 
a = N/L  #Distance between each slice    
h = 10**(-18)  #Step [s]


def psi_0(x): #Return the initial wave function psi (x,0) at time t = 0 
    return np.exp(-(x-x_0)**2/2*sigma**2)*np.exp(1j*kappa*x) 


def initial_condition(N, L, h): #Set a wave function at time t = 0, with each point in x separated by a
    x = np.linspace(0, L, N+1) #Each point separated by a
    psi = np.array(psi_0(x)) #Initial condition  
    psi[0] = 0 #The wave function equal 0 at discontinuity point i.e. infinite potential value 
    psi[-1] = 0 #Same as last 
    return psi 
    
    
def v(psi): #return the vector v
    b1 = 1 - 1j * h * hbar / (2 * M * a ** 2)
    b2 = h * hbar * 1j / (4 * M * a ** 2)
    
    v = np.zeros(len(psi), dtype = 'complex_') #Initialize the vector v 
    n = len(psi) - 1 #Last element of vector v 
    
    v[0] = b1 * psi[0] + b2 * (psi[1])
    v[1:n] = b1 * psi[1:n] + b2 * (psi[2:n+1] + psi[0:n-1])
    v[n] = b1 * psi[n] + b2 * (psi[n-1])
    
    return v

psi = initial_condition(N, L, h)
v = v(psi)
print(v)

[-1.38754903e-45+2.53989147e-45j  8.77582562e-01+4.79425539e-01j
  5.40302306e-01+8.41470985e-01j ... -8.71162202e-01+4.90995334e-01j
 -9.99912459e-01+1.32315347e-02j -3.82945873e-47-2.89393754e-45j]


### Question 2
L'objet de ce numéro est de résoudre le système $\mathbf{v} = \mathbf{A}\mathbf{x}$ afin d'obtenir le vecteur $\mathbf{x}$. Pour ce faire l'algorithme de Thomas qui est une forme simplifiée de l'élimination gaussienne. On va premièrement écrire le vecteur sous la forme $$a_{i} x_{i-1}+b_{i} x_{i}+c_{i} x_{i+1}=d_{i}.$$  

In [58]:
def A_tri(N):
    a1 = 1 + 1j * h * hbar / (2 * M * a ** 2)
    a2 = - h * hbar * 1j / (4 * M * a ** 2)
    A = np.zeros([3, N + 1], dtype = 'complex') #We write the tridiagonal matrix in the good form 
    
    A[0, 0] = 0
    A[0, 1:] = a2
    A[1, :] = a1
    A[2, 0:N] = a2
    A[2, N] = 0
    return A 

A_tri = A_tri(N)  

Ensuite, on utilise l'algorithme de Thomas pour résoudre le système. Brièvement, on invente des nouveaux coefficients $c_i^\prime$ et $d_i^\prime$ qui ont les valeurs suivantes:   
$$
c_{i}^{\prime}=\left\{\begin{array}{ll}
\frac{c_{i}}{b_{i}} & ; \quad i=1 \\
\frac{c_{i}}{b_{i}-a_{i} c_{i-1}^{\prime}} & ; \quad i=2,3, \ldots, n-1
\end{array}\right.
$$
et
$$
d_{i}^{\prime}=\left\{\begin{array}{ll}
\frac{d_{i}}{b_{i}} & ; \quad i=1 \\
\frac{d_{i}-a_{i} d_{i-1}^{\prime}}{b_{i}-a_{i} c_{i-1}^{\prime}} & ; \quad i=2,3, \ldots, n
\end{array}\right.
$$


On obtient alors la solution en utilisant les nouveaux coefficients: 
$$x_{n}=d_{n}^{\prime}$$
et 
$$
x_{i}=d_{i}^{\prime}-c_{i}^{\prime} x_{i+1} \quad ; i=n-1, n-2, \ldots, 1 .
$$ 

In [59]:
def Thomas(A_tri, v): 
    """
    The function returns the vector x of the system of equation Ax = v 
    param 1 A: is a triadiagonal matrix written in the form a_{i} * x_{i-1} + b_{i} * x_{i}+ c_{i} * x_{i+1}
    param 2 v: v is the vector of the RHS    
    return: x, the solution of the system  
    """
    N = len(v) - 1 
    c_prime = np.zeros(N, dtype = 'complex') #Initialization of the vector c' 
    d_prime = np.zeros(N + 1, dtype = 'complex') #Initialization of the vector d' 
    x = np.zeros(N + 1, dtype = 'complex') #Initialization of the vector x 
    
    a_ = A_tri[0, :] #Lower diagonal
    b_ = A_tri[1, :] #Middle diagonal
    c_ = A_tri[2, :] #Upper diagonal
    d_ = v[:] #RHS 
    
    c_prime[0] = c_[0]/b_[0] #First element of the new c' vector
    d_prime[0] = d_[0]/b_[0] #First element of the new d' vector
    
    for i in range(1, N):   
        c_prime[i] = c_[i]/(b_[i] - a_[i] * c_prime[i - 1]) #New values of c' 
    
    for i in range(1, N+1): 
        d_prime[i] = (d_[i] - a_[i] * d_prime[i-1])/(b_[i] - a_[i] * c_prime[i-1]) #New values of d'
    
    x[N] = d_prime[N] #Last element of the vector x 
    
    for i in range(N-1, -1, -1):
        x[i] = d_prime[i] - c_prime[i]*x[i+1] #New values of x by substitution of c' and d', from the before last to the first element

        
    return x


On peut vérifier si l'algorithme est fonctionnel en vérifiant si le produit matricielle et entre la matrice $\mathbf{A}$ et le vecteur $\mathbf{x}$ donne bien le vecteur $\mathbf{v}$. 

In [65]:
#First we must write the matrix A in its matrix form 

def A(N):
    
    A = np.zeros([N+1, N+1], dtype = 'complex') #We initialize the size of the square matrix
    a1 = 1 + 1j * h * hbar / (2 * M * a ** 2)
    a2 = - h * hbar * 1j / (4 * M * a ** 2)
    
    for i in range(N+1): #Middle diagonal
        A[i, i] = a1
    
    for i in range(N):   #Upper diagonal  
        A[i, i+1] = a2 
    
    for i in range(N):   #Lower diagonal
        A[i+1, i] = a2  
    
    return A

A = A(N)

#We check if the matrix product Ax = v 

x = Thomas(A_tri, v) #The vector x found by Thomas algorithm 
Ax = np.matmul(A, x) #The matrix product

comparison = Ax == v
equal_arrays = comparison.all() #If the value of equal_arrays is true then all the elements of Ax and v are the same 
if equal_arrays is True:
    print("La multiplication matricielle Ax est égal au vecteur v.")

### Question 3


### Question 4


### Question 5

## Références
1. Newman, Mark. Computational physics. CreateSpace Independent Publ., 2013.